In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import joblib
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
#from sklearn.preprocessing import StandardScaler


%store -r X_features
%store -r X_features_poly
%store -r y_labels

%store -r totaldf
totaldf = totaldf


In [2]:
voting_reg = joblib.load("voting_reg.pkl")
poly_ridge_reg = joblib.load("poly_ridge_reg.pkl")
svm_rbf_reg = joblib.load("svm_rbf_reg.pkl")
forest_reg = joblib.load("forest_regressor.pkl")
seq_model = keras.models.load_model('D:\python\\partial data\\first')



In [3]:
totaldf.insert(5,'poly ridge μ', totaldf.μob*(np.exp(np.power(poly_ridge_reg.predict(X_features_poly[2]),2)) -0.0012))
totaldf.insert(6,'rbf μ', totaldf.μob*(np.exp(np.power(svm_rbf_reg.predict(X_features[2]),2)) -0.0012))
totaldf.insert(7,'forest μ', totaldf.μob*(np.exp(np.power(forest_reg.predict(X_features[2]),2)) -0.0012))
totaldf.insert(8,'voting μ', totaldf.μob*(np.exp(np.power(voting_reg.predict(X_features[2]),2)) -0.0012))
totaldf.insert(9,'sequential', (np.exp(np.power(seq_model.predict(X_features[2]),2)) -0.0012))
totaldf.sequential = totaldf.sequential*totaldf.μob
totaldf.insert(10,'Beal', totaldf['μob'] +(0.001*totaldf['ΔP'])*(0.024*np.power(totaldf['μob'] ,1.6)+0.038*np.power(totaldf['μob'] ,0.56)))
totaldf.insert(11,'Kouzel modified',totaldf['μob']*np.exp((-2.34864*np.power(10,-5.0)+9.30705*np.power(10,-5.0)*np.power(totaldf['μob'],0.181))*totaldf.ΔP))

72/72 [==============================] - 0s 2ms/step


In [4]:
def Stats_matrix(a):
    average_error = []
    standard_deviation = []
    absolute_average_error = []
    absolute_standard_deviation =[]
    for i in range(7):
        average_error.append(np.average((a.iloc[:,i+5]-a.μ)/a.μ*100))
        standard_deviation.append(np.std((a.iloc[:,i+5]-a.μ)/a.μ*100))
        absolute_average_error.append(np.average(np.absolute((a.iloc[:,i+5]-a.μ)/a.μ*100)))
        absolute_standard_deviation.append(np.std(np.absolute((a.iloc[:,i+5]-a.μ)/totaldf.μ*100)))
    statistics  = pd.DataFrame(list(zip(average_error,standard_deviation,absolute_average_error,
                                        absolute_standard_deviation)),columns = ['% AE','% Std Dev AE','%AAE','% Std Dev AAE'],index = ['Ridge Regression','SVM RBF kernel','Random Forest','Voting Regressor','ANN with 1 layer','Beal','Kouzel'])
    return statistics

Stats_matrix(totaldf)

,% AE,% Std Dev AE,%AAE,% Std Dev AAE
Ridge Regression,-0.472932,5.881186,3.606496,4.669604
SVM RBF kernel,-0.794406,5.763032,3.475656,4.665130
Random Forest,-0.303854,5.549532,3.493291,4.322794
Voting Regressor,-0.580324,5.574630,3.402190,4.454030
ANN with 1 layer,-1.023799,5.710593,3.438392,4.672954
Beal,-2.175139,5.988947,3.832978,5.089891
Kouzel,-2.154380,6.641349,4.222523,5.560501
